# Pollen Challenge ICPR 2020

## Classification

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import json

* Class 1: 1567 Images
* Class 2: 773 Images
* Class 3: 8216 Images
* Class 4: 724 Images

### Training - Normal images with VGG16

In [ ]:
image_names = []
image_labels = []
for i in range(1, 5):
    input_dir = f"./train/images/{i}/train_SEGM/"
    images = os.listdir(input_dir)
    for j in range(len(images)):
        if images[j].split(".")[-1] == "png":
            image_names.append(input_dir + images[j])
            image_labels.append(i - 1)

All images are 84 x 84

In [ ]:
dim = (96, 96)

In [ ]:
images = []
for image in image_names:
    if image.split('.')[-1] == 'png':
        images.append(cv2.resize(cv2.imread(image, cv2.IMREAD_UNCHANGED), dim, interpolation=cv2.INTER_AREA))

In [ ]:
images = np.array(images).astype(np.float32)
images = images / 255.
image_labels = np.array(image_labels)
image_labels = to_categorical(image_labels, num_classes=4)

In [ ]:
model = keras.applications.VGG16(include_top=False, input_shape=(96, 96, 3))

In [ ]:
for layer in model.layers[:25]:
    layer.trainable = False
flatten_out = Flatten()(model.output)
class_out = Dense(512, activation='relu')(flatten_out)
class_out = Dropout(0.5)(class_out)
output = Dense(4, activation='softmax')(class_out)
model = keras.Model(inputs=model.inputs, outputs=output)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, image_labels, test_size=0.1, random_state=42)

In [ ]:
pd.value_counts(np.argmax(y_train, axis=1))

In [ ]:
pd.value_counts(np.argmax(y_val, axis=1))

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(X_val, y_val), shuffle=True)

In [ ]:
model.save("vgg16_normalised_96_noaug_lastconvtop_10.h5")

In [ ]:
y_pred = np.argmax(model.predict(X_val.astype(np.float32)), axis=1)
y_val_ = np.argmax(y_val, axis=1)
print(classification_report(y_val_, y_pred))
print(confusion_matrix(y_val_, y_pred))

### Training - Segmented images with VGG16

In [ ]:
image_names = []
image_masks = []
image_labels = []
for i in range(1, 5):
    input_dir = f"./train/images/{i}/train_OBJ/"
    mask_dir = f"./train/images/{i}/train_MASK/"
    images = os.listdir(input_dir)
    for j in range(len(images)):
        if images[j].split(".")[-1] == "png":
            image_names.append(input_dir + images[j])
            image_masks.append(mask_dir + images[j].replace("OBJ", "MASK"))
            image_labels.append(i - 1)

In [ ]:
images = []
for i in tqdm(range(len(image_names))):
    if image_names[i].split('.')[-1] == 'png':
        img = cv2.imread(image_names[i], cv2.IMREAD_UNCHANGED)
        m = cv2.imread(image_masks[i], cv2.IMREAD_GRAYSCALE)
        res = cv2.bitwise_and(img, img, mask=m)
        images.append(cv2.resize(res, dim, interpolation=cv2.INTER_AREA))

In [ ]:
images = np.array(images).astype(np.float32)
images = images / 255.
image_labels = np.array(image_labels)
image_labels = to_categorical(image_labels, num_classes=4)

In [ ]:
model = keras.applications.VGG16(include_top=False, input_shape=(96, 96, 3))

In [ ]:
for layer in model.layers[:25]:
    layer.trainable = False
flatten_out = Flatten()(model.output)
class_out = Dense(512, activation='relu')(flatten_out)
class_out = Dropout(0.5)(class_out)
output = Dense(4, activation='softmax')(class_out)
model = keras.Model(inputs=model.inputs, outputs=output)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, image_labels, test_size=0.1, random_state=42)

In [ ]:
pd.value_counts(np.argmax(y_train, axis=1))

In [ ]:
pd.value_counts(np.argmax(y_val, axis=1))

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(X_val, y_val), shuffle=True)

In [ ]:
model.save("vgg16_segmented_96_noaug_lastconvtop_10.h5")

In [ ]:
y_pred = np.argmax(model.predict(X_val.astype(np.float32)), axis=1)
y_val_ = np.argmax(y_val, axis=1)
print(classification_report(y_val_, y_pred))
print(confusion_matrix(y_val_, y_pred))

### Testing

In [ ]:
test_image_names = []
test_input_dir = f"./test/images/"
test_images = os.listdir(test_input_dir)
for j in range(len(test_images)):
    if test_images[j].split(".")[-1] == "png":
        test_image_names.append(test_input_dir + test_images[j])

In [ ]:
test_images = []
for test_image in test_image_names:
    if test_image.split('.')[-1] == 'png':
        test_images.append(cv2.resize(cv2.imread(test_image, cv2.IMREAD_UNCHANGED),
                                      dim, interpolation=cv2.INTER_AREA))

In [ ]:
test_images = np.array(test_images).astype(np.float32)
test_images = test_images / 255.

In [ ]:
model = keras.models.load_model("vgg16_normalised_96_noaug_lastconvtop_10.h5")

In [ ]:
test_pred = np.argmax(model.predict(test_images), axis=1)

In [ ]:
pd.value_counts(test_pred)

In [ ]:
submission = []
for i in range(len(test_image_names)):
    submission.append({"Filename": "{}".format(test_image_names[i].split("/")[-1]),
                       "Class": str(test_pred[i] + 1)
                      })

In [ ]:
with open('submission_obj_trainonly_vgg16_96.json', 'w', encoding='utf-8') as f:
    json.dump(submission, f, ensure_ascii=False, indent=4)